<a href="https://colab.research.google.com/github/41371113h-xian/114-1/blob/main/hw_3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:

!pip install -q gradio pandas matplotlib python-dateutil gspread gspread_dataframe google-auth


In [4]:
# ===========================
# ToDo + Pomodoro — Gradio v4 全功能 + Google Sheets 同步（Colab 版完整版）
# ===========================
import io, os, json, time, threading
from datetime import datetime

import pandas as pd
import matplotlib.pyplot as plt
import gradio as gr
from dateutil.parser import parse as dtparse

# Google Sheets 依賴
import gspread
from gspread_dataframe import set_with_dataframe
from google.auth import default as google_auth_default

# -------- 常數 / 工具 --------
STATE_FILE = "todo_state.json"
DATE_FMT = "%Y-%m-%d"
DT_FMT = "%Y-%m-%d %H:%M:%S"

PRIORITIES = ["高", "中", "低"]
STATUSES = ["未開始", "進行中", "已完成", "已暫停"]

def now_str():
    return datetime.now().strftime(DT_FMT)

def safe_date_str(s):
    if not s:
        return ""
    try:
        return dtparse(str(s)).date().strftime(DATE_FMT)
    except Exception:
        return ""

def ensure_int(x, default=0):
    try:
        if isinstance(x, float) and pd.isna(x):
            return default
        return int(float(x))
    except Exception:
        return default

def parse_tags(s):
    s = (s or "").strip()
    if not s:
        return []
    raw = [x.strip() for x in s.replace(" "," ").replace("　"," ").replace("，",",").split(",")]
    return [x for x in raw if x]

def tags_to_str(tags):
    if not tags:
        return ""
    return ", ".join(tags)

# -------- 狀態 --------
def new_state():
    return {
        "tasks": [],
        "next_id": 1,
        "active": {
            "task_id": None,
            "running": False,
            "paused": False,
            "total_sec": 0,
            "left_sec": 0,
            "mode": "focus",  # focus / break_short / break_long
            "cycle_idx": 0,
            "stop_flag": False,
            "auto_next": True,
            "focus_min": 25,
            "short_break_min": 5,
            "long_break_min": 15,
            "long_every": 4,
            "sound_on": True
        },
        "settings": {
            "auto_complete_when_reach_est": True
        },
        "logs": []
    }

STATE = new_state()
LOCK = threading.Lock()

# -------- 永續化 --------
def save_state(path=STATE_FILE):
    with LOCK:
        with open(path, "w", encoding="utf-8") as f:
            json.dump(STATE, f, ensure_ascii=False, indent=2)

def load_state(path=STATE_FILE):
    if not os.path.exists(path):
        return False
    try:
        with open(path, "r", encoding="utf-8") as f:
            data = json.load(f)
        for t in data.get("tasks", []):
            t["id"] = ensure_int(t.get("id"))
            t["est_pomodoros"] = ensure_int(t.get("est_pomodoros"), 1)
            t["used_pomodoros"] = ensure_int(t.get("used_pomodoros"))
            t["time_tracked_sec"] = ensure_int(t.get("time_tracked_sec"))
            t.setdefault("priority", "中")
            t.setdefault("status", "未開始")
            t.setdefault("desc", "")
            t["tags"] = t.get("tags", [])
        with LOCK:
            STATE.clear()
            STATE.update(data)
        return True
    except Exception as e:
        print("載入狀態失敗：", e)
        return False

load_state()

# -------- DataFrame / 視覺化 --------
def df_from_tasks(tasks=None):
    tasks = STATE["tasks"] if tasks is None else tasks
    cols = [
        "id","name","desc","status","priority","created_at","due_date","completed_at",
        "est_pomodoros","used_pomodoros","time_tracked_sec","tags"
    ]
    if not tasks:
        return pd.DataFrame(columns=cols)
    rows = []
    for t in tasks:
        rows.append({
            "id": t["id"],
            "name": t["name"],
            "desc": t.get("desc", ""),
            "status": t.get("status", "未開始"),
            "priority": t.get("priority", "中"),
            "created_at": t.get("created_at", ""),
            "due_date": t.get("due_date", ""),
            "completed_at": t.get("completed_at", ""),
            "est_pomodoros": t.get("est_pomodoros", 1),
            "used_pomodoros": t.get("used_pomodoros", 0),
            "time_tracked_sec": t.get("time_tracked_sec", 0),
            "tags": tags_to_str(t.get("tags", []))
        })
    return pd.DataFrame(rows, columns=cols)

def df_from_logs():
    cols = ["task_id","task_name","type","start","end","duration_sec","cycle_no"]
    if not STATE["logs"]:
        return pd.DataFrame(columns=cols)
    return pd.DataFrame(STATE["logs"], columns=cols)

def build_all_figures(df_tasks, df_logs):
    figs = []

    # 圖1：完成比例
    fig1 = plt.figure(figsize=(4.5, 4), dpi=120)
    if df_tasks.empty:
        plt.text(0.5, 0.5, "尚無任務", ha="center", va="center"); plt.axis("off")
    else:
        stat = df_tasks["status"].value_counts()
        plt.pie(stat, labels=stat.index, autopct="%1.1f%%", startangle=90)
        plt.title("任務完成比例")
    figs.append(fig1)

    # 圖2：各任務番茄數
    fig2 = plt.figure(figsize=(6, 4), dpi=120)
    if df_tasks.empty:
        plt.text(0.5, 0.5, "尚無任務", ha="center", va="center"); plt.axis("off")
    else:
        used = df_tasks.assign(used_pomodoros=pd.to_numeric(df_tasks["used_pomodoros"], errors="coerce").fillna(0))
        plt.bar(used["name"], used["used_pomodoros"])
        plt.xticks(rotation=45, ha="right")
        plt.title("各任務番茄數"); plt.tight_layout()
    figs.append(fig2)

    # 圖3：每日番茄趨勢
    fig3 = plt.figure(figsize=(6, 4), dpi=120)
    if df_logs.empty:
        plt.text(0.5, 0.5, "尚無番茄紀錄", ha="center", va="center"); plt.axis("off")
    else:
        fl = df_logs[df_logs["type"] == "focus"].copy()
        if fl.empty:
            plt.text(0.5, 0.5, "尚無番茄紀錄", ha="center", va="center"); plt.axis("off")
        else:
            fl["day"] = pd.to_datetime(fl["end"]).dt.date
            trend = fl.groupby("day")["duration_sec"].sum().reset_index()
            trend["pomodoros"] = trend["duration_sec"] / 1500.0  # 25 分 = 1500 秒
            plt.plot(trend["day"], trend["pomodoros"], marker="o")
            plt.title("每日番茄趨勢（換算番茄數）")
            plt.xlabel("日期"); plt.ylabel("番茄數"); plt.grid(True, alpha=0.3)
    figs.append(fig3)

    return figs

def refresh_all():
    df_tasks = df_from_tasks()
    df_logs = df_from_logs()
    figs = build_all_figures(df_tasks, df_logs)
    return df_tasks, df_logs, figs[0], figs[1], figs[2]

# -------- Google Sheets 連線 / 同步 --------
GS = {
    "url": "https://docs.google.com/spreadsheets/d/1F_Mv1itqC82b-xclRF0mKnAew1dvX_SuTDoJ15oLNBQ/edit?usp=sharing",
    "client": None,
    "sh": None,
    "ws_tasks": None,
    "ws_logs": None,
    "auto_sync": True
}

TASK_HEADERS = [
    "id","name","desc","status","priority","created_at","due_date","completed_at",
    "est_pomodoros","used_pomodoros","time_tracked_sec","tags"
]
LOG_HEADERS = ["task_id","task_name","type","start","end","duration_sec","cycle_no"]

def gsheet_connect(url):
    try:
        creds, _ = google_auth_default()
        creds = creds.with_scopes([
            "https://www.googleapis.com/auth/spreadsheets",
            "https://www.googleapis.com/auth/drive"
        ])
        gc = gspread.authorize(creds)
        sh = gc.open_by_url(url)

        def get_or_create(title, headers):
            try:
                ws = sh.worksheet(title)
                first_row = ws.row_values(1)
                if first_row != headers:
                    ws.clear()
                    ws.append_row(headers)
                return ws
            except gspread.WorksheetNotFound:
                ws = sh.add_worksheet(title=title, rows="1000", cols="26")
                ws.append_row(headers)
                return ws

        ws_tasks = get_or_create("tasks", TASK_HEADERS)
        ws_logs = get_or_create("logs", LOG_HEADERS)

        GS.update({"url": url, "client": gc, "sh": sh, "ws_tasks": ws_tasks, "ws_logs": ws_logs})
        return "✅ 已連線 Google 試算表並建立/校正工作表欄位"
    except Exception as e:
        return f"❌ 連線失敗：{e}"

def gsheet_sync_tasks():
    if not GS.get("ws_tasks"):
        return "⚠️ 尚未連線 Google 試算表"
    df = df_from_tasks()
    if "tags" in df.columns:
        df["tags"] = df["tags"].astype(str)
    set_with_dataframe(GS["ws_tasks"], df, include_index=False, include_column_header=True, resize=True)
    return f"📤 已同步 {len(df)} 筆任務至工作表 tasks"

def gsheet_sync_logs():
    if not GS.get("ws_logs"):
        return "⚠️ 尚未連線 Google 試算表"
    df = df_from_logs()
    set_with_dataframe(GS["ws_logs"], df, include_index=False, include_column_header=True, resize=True)
    return f"📤 已同步 {len(df)} 筆日誌至工作表 logs"

def gsheet_auto_sync_if_enabled():
    if GS.get("auto_sync", False) and GS.get("ws_tasks"):
        try:
            gsheet_sync_tasks()
            gsheet_sync_logs()
        except Exception:
            pass

# -------- 查詢 / 篩選 --------
def search_and_filter(q, status, priority, tag_kw, due_from, due_to):
    df = df_from_tasks()
    if not df.empty:
        q = (q or "").strip().lower()
        if q:
            mask = (
                df["name"].str.lower().str.contains(q, na=False) |
                df["desc"].str.lower().str.contains(q, na=False)
            )
            df = df[mask]
        if status and status != "全部":
            df = df[df["status"] == status]
        if priority and priority != "全部":
            df = df[df["priority"] == priority]
        tag_kw = (tag_kw or "").strip().lower()
        if tag_kw:
            df = df[df["tags"].str.lower().str.contains(tag_kw, na=False)]
        if due_from:
            df = df[(df["due_date"] >= safe_date_str(due_from)) | (df["due_date"] == "")]
        if due_to:
            df = df[(df["due_date"] <= safe_date_str(due_to)) | (df["due_date"] == "")]
    figs = build_all_figures(df_from_tasks(), df_from_logs())
    return df.reset_index(drop=True), figs[0], figs[1], figs[2]

# -------- CRUD --------
def add_task(name, desc, priority, due_date, est, tags):
    name = (name or "").strip()
    if not name:
        return *refresh_all(), "❗請輸入任務名稱"
    t = {
        "id": STATE["next_id"],
        "name": name,
        "desc": desc or "",
        "status": "未開始",
        "priority": priority if priority in PRIORITIES else "中",
        "created_at": now_str(),
        "due_date": safe_date_str(due_date),
        "completed_at": "",
        "est_pomodoros": max(1, ensure_int(est, 1)),
        "used_pomodoros": 0,
        "time_tracked_sec": 0,
        "tags": parse_tags(tags)
    }
    with LOCK:
        STATE["tasks"].append(t)
        STATE["next_id"] += 1
        save_state()
    gsheet_auto_sync_if_enabled()
    return *refresh_all(), f"✅ 已新增：{t['name']}（ID {t['id']}）"

def edit_task(tid, name, desc, priority, due_date, est, status, tags):
    tid = ensure_int(tid, -1)
    with LOCK:
        t = next((x for x in STATE["tasks"] if x["id"] == tid), None)
        if not t:
            return *refresh_all(), "❌ 找不到任務 ID"
        if name:
            t["name"] = name
        t["desc"] = desc or t.get("desc", "")
        if priority in PRIORITIES:
            t["priority"] = priority
        t["due_date"] = safe_date_str(due_date) or t.get("due_date", "")
        t["est_pomodoros"] = max(1, ensure_int(est, t.get("est_pomodoros", 1)))
        if status in STATUSES:
            t["status"] = status
        t["tags"] = parse_tags(tags) if tags is not None else t.get("tags", [])
        save_state()
    gsheet_auto_sync_if_enabled()
    return *refresh_all(), f"✏️ 已更新任務 {tid}"

def delete_task(tid):
    tid = ensure_int(tid, -1)
    with LOCK:
        before = len(STATE["tasks"])
        STATE["tasks"] = [x for x in STATE["tasks"] if x["id"] != tid]
        save_state()
    gsheet_auto_sync_if_enabled()
    msg = "🗑️ 已刪除" if len(STATE["tasks"]) < before else "❌ 找不到該任務 ID"
    return *refresh_all(), msg

def bulk_delete(ids_str):
    ids = [ensure_int(x.strip()) for x in (ids_str or "").replace("，", ",").split(",") if x.strip()]
    with LOCK:
        before = len(STATE["tasks"])
        STATE["tasks"] = [x for x in STATE["tasks"] if x["id"] not in ids]
        save_state()
    gsheet_auto_sync_if_enabled()
    return *refresh_all(), f"🗑️ 已刪除 {before - len(STATE['tasks'])} 筆"

def mark_done(tid):
    tid = ensure_int(tid, -1)
    with LOCK:
        t = next((x for x in STATE["tasks"] if x["id"] == tid), None)
        if not t:
            return *refresh_all(), "❌ 找不到任務 ID"
        t["status"] = "已完成"
        t["completed_at"] = now_str()
        save_state()
    gsheet_auto_sync_if_enabled()
    return *refresh_all(), f"🎉 任務 {tid} 已完成"

def bulk_done(ids_str):
    ids = [ensure_int(x.strip()) for x in (ids_str or "").replace("，", ",").split(",") if x.strip()]
    c = 0
    with LOCK:
        for t in STATE["tasks"]:
            if t["id"] in ids:
                t["status"] = "已完成"
                t["completed_at"] = now_str()
                c += 1
        save_state()
    gsheet_auto_sync_if_enabled()
    return *refresh_all(), f"✅ 已標記完成 {c} 筆"

# -------- 番茄鐘（Streaming） --------
def start_timer(tid, focus, short_break, long_break, long_every, auto_next, sound_on):
    tid = ensure_int(tid, -1)
    task = next((x for x in STATE["tasks"] if x["id"] == tid), None)
    if not task:
        yield "❌ 找不到任務 ID", 0, ""
        return
    if STATE["active"]["running"]:
        yield "⏳ 已有計時在進行中，請先停止或暫停", STATE["active"]["left_sec"], ""
        return

    focus_s = max(5, int(float(focus) * 60))
    short_s = max(5, int(float(short_break) * 60))
    long_s = max(5, int(float(long_break) * 60))

    with LOCK:
        STATE["active"].update({
            "task_id": tid,
            "running": True,
            "paused": False,
            "stop_flag": False,
            "mode": "focus",
            "total_sec": focus_s,
            "left_sec": focus_s,
            "cycle_idx": 0,
            "auto_next": bool(auto_next),
            "focus_min": float(focus),
            "short_break_min": float(short_break),
            "long_break_min": float(long_break),
            "long_every": ensure_int(long_every, 4),
            "sound_on": bool(sound_on)
        })
        save_state()

    def html(title, total, left, sound=False):
        pct = 0 if total == 0 else (total - left) / total
        audio = """
        <audio autoplay>
          <source src="data:audio/wav;base64,UklGRiQAAABXQVZFZm10IBAAAAABAAEAESsAACJWAAACABAAZGF0YQgAAAAA" type="audio/wav">
        </audio>
        """ if sound and STATE["active"].get("sound_on", True) else ""
        return f"""
        <div style="border:1px solid #ddd; width:100%; height:18px; border-radius:6px;">
          <div style="height:100%; width:{pct*100:.2f}%; background:#4CAF50; border-radius:6px;"></div>
        </div>
        <div style="margin-top:6px; font-size:14px;"><b>{title}</b>　剩餘 {left//60:02d}:{left%60:02d}</div>
        {audio}
        """

    while True:
        # 專注
        STATE["active"]["mode"] = "focus"
        start_ts = now_str()
        total = STATE["active"]["total_sec"] = focus_s
        left = STATE["active"]["left_sec"] = focus_s
        while left >= 0:
            if STATE["active"]["stop_flag"]:
                STATE["active"]["running"] = False
                yield "⏹️ 計時已停止", left, html("專注中", total, max(0, left))
                save_state()
                return
            if STATE["active"]["paused"]:
                time.sleep(0.2); continue
            yield f"🍅 任務 {task['id']}：{task['name']} 專注中…", left, html("專注中", total, left)
            time.sleep(1); left -= 1; STATE["active"]["left_sec"] = left

        # 專注結束：記錄 +1 番茄
        end_ts = now_str()
        duration = total
        with LOCK:
            task["used_pomodoros"] = ensure_int(task.get("used_pomodoros"), 0) + 1
            if task.get("status") != "已完成":
                task["status"] = "進行中"
            task["time_tracked_sec"] = ensure_int(task.get("time_tracked_sec"), 0) + duration
            STATE["active"]["cycle_idx"] += 1
            cycle_no = STATE["active"]["cycle_idx"]
            STATE["logs"].append({
                "task_id": task["id"], "task_name": task["name"],
                "type": "focus", "start": start_ts, "end": end_ts,
                "duration_sec": int(duration), "cycle_no": int(cycle_no)
            })
            if STATE["settings"].get("auto_complete_when_reach_est", True):
                if task["used_pomodoros"] >= task["est_pomodoros"] and task["status"] != "已完成":
                    task["status"] = "已完成"; task["completed_at"] = now_str()
            save_state()
        gsheet_auto_sync_if_enabled()
        yield f"⏰ 專注結束！+1 番茄（第 {STATE['active']['cycle_idx']} 顆）", 0, html("完成專注", 1, 0, sound=True)

        # 休息
        use_long = (STATE["active"]["long_every"] > 0 and STATE["active"]["cycle_idx"] % STATE["active"]["long_every"] == 0)
        STATE["active"]["mode"] = "break_long" if use_long else "break_short"
        b_total = long_s if use_long else short_s
        b_left = b_total
        while b_left >= 0:
            if STATE["active"]["stop_flag"]:
                STATE["active"]["running"] = False
                yield "⏹️ 計時已停止（休息中）", b_left, html("休息中", b_total, max(0, b_left))
                save_state()
                return
            if STATE["active"]["paused"]:
                time.sleep(0.2); continue
            yield f"🫖 {'長休中' if use_long else '短休中'}…", b_left, html('長休中' if use_long else '短休中', b_total, b_left)
            time.sleep(1); b_left -= 1

        yield "✅ 休息結束", 0, html("休息完成", 1, 0, sound=True)
        if not STATE["active"]["auto_next"]:
            STATE["active"]["running"] = False; save_state(); return

def pause_timer():
    if not STATE["active"]["running"]:
        return "ℹ️ 目前沒有進行中的計時", STATE["active"]["left_sec"], ""
    STATE["active"]["paused"] = True; save_state()
    return "⏸️ 已暫停", STATE["active"]["left_sec"], ""

def resume_timer():
    if not STATE["active"]["running"]:
        return "ℹ️ 目前沒有進行中的計時", 0, ""
    if not STATE["active"]["paused"]:
        return "ℹ️ 計時未暫停", STATE["active"]["left_sec"], ""
    STATE["active"]["paused"] = False; save_state()
    return "▶️ 繼續", STATE["active"]["left_sec"], ""

def stop_timer():
    if not STATE["active"]["running"]:
        return "ℹ️ 目前沒有進行中的計時", 0, ""
    STATE["active"]["stop_flag"] = True; save_state()
    return "🛑 已發送停止指令", STATE["active"]["left_sec"], ""

def toggle_auto_complete(flag):
    STATE["settings"]["auto_complete_when_reach_est"] = bool(flag); save_state()
    return "⚙️ 已更新設定"

# -------- 匯出 / 匯入 --------
def export_json(reindent=True):
    data = STATE
    b = io.BytesIO()
    s = json.dumps(data, ensure_ascii=False, indent=(2 if reindent else None))
    b.write(s.encode("utf-8")); b.seek(0)
    fn = f"todo_full_{datetime.now().strftime('%Y%m%d_%H%M%S')}.json"
    return gr.File.update(value=(b, fn), visible=True), "💾 已匯出 JSON"

def export_csv():
    df = df_from_tasks()
    b = io.BytesIO()
    df.to_csv(b, index=False, encoding="utf-8-sig")
    b.seek(0)
    fn = f"todo_tasks_{datetime.now().strftime('%Y%m%d_%H%M%S')}.csv"
    return gr.File.update(value=(b, fn), visible=True), "💾 已匯出 CSV（任務表）"

def import_json(file, mode):
    if file is None:
        return *refresh_all(), "❗未選擇檔案"
    try:
        payload = json.load(open(file.name, "r", encoding="utf-8"))
        if mode == "覆蓋":
            with LOCK:
                STATE.clear(); STATE.update(payload); save_state()
        else:
            incoming_tasks = payload.get("tasks", [])
            incoming_logs = payload.get("logs", [])
            with LOCK:
                ids_exist = {t["id"] for t in STATE["tasks"]}
                for t in incoming_tasks:
                    if t["id"] in ids_exist: continue
                    STATE["tasks"].append(t)
                STATE["logs"].extend(incoming_logs)
                STATE["next_id"] = max(STATE["next_id"], payload.get("next_id", STATE["next_id"]))
                save_state()
        gsheet_auto_sync_if_enabled()
        return *refresh_all(), "📥 匯入完成"
    except Exception as e:
        return *refresh_all(), f"❌ 匯入失敗：{e}"

def import_csv(file, mode):
    if file is None:
        return *refresh_all(), "❗未選擇檔案"
    try:
        df = pd.read_csv(file.name, dtype=str)
        out, max_id = [], 0
        for _, r in df.iterrows():
            t = {
                "id": ensure_int(r.get("id")),
                "name": str(r.get("name", "")),
                "desc": str(r.get("desc", "")),
                "status": str(r.get("status", "未開始")),
                "priority": str(r.get("priority", "中")),
                "created_at": str(r.get("created_at", now_str())),
                "due_date": safe_date_str(r.get("due_date", "")),
                "completed_at": str(r.get("completed_at", "")),
                "est_pomodoros": ensure_int(r.get("est_pomodoros"), 1),
                "used_pomodoros": ensure_int(r.get("used_pomodoros"), 0),
                "time_tracked_sec": ensure_int(r.get("time_tracked_sec"), 0),
                "tags": parse_tags(r.get("tags", ""))
            }
            max_id = max(max_id, t["id"])
            out.append(t)
        with LOCK:
            if mode == "覆蓋":
                STATE["tasks"] = out; STATE["next_id"] = max_id + 1
            else:
                ids_exist = {t["id"] for t in STATE["tasks"]}
                for t in out:
                    if t["id"] in ids_exist: continue
                    STATE["tasks"].append(t)
                STATE["next_id"] = max(STATE["next_id"], max_id + 1)
            save_state()
        gsheet_auto_sync_if_enabled()
        return *refresh_all(), f"📥 已從 CSV 匯入 {len(out)} 筆（{mode}）"
    except Exception as e:
        return *refresh_all(), f"❌ 匯入失敗：{e}"

# -------- 介面 --------
with gr.Blocks(title="ToDo + Pomodoro — 全功能版 + Google Sheets") as demo:
    gr.Markdown("# ✅ ToDo + 🍅 Pomodoro（Gradio v4）＋ Google Sheets 同步")

    # 任務清單
    with gr.Tab("任務清單"):
        with gr.Row():
            q = gr.Textbox(label="關鍵字（名稱/描述）")
            status = gr.Dropdown(["全部"] + STATUSES, value="全部", label="狀態")
            priority = gr.Dropdown(["全部"] + PRIORITIES, value="全部", label="優先度")
        with gr.Row():
            tag_kw = gr.Textbox(label="標籤關鍵字")
            due_from = gr.Textbox(label="到期日自（YYYY-MM-DD）")
            due_to = gr.Textbox(label="到期日至（YYYY-MM-DD）")
            btn_query = gr.Button("查詢 / 篩選", variant="primary")
            btn_show_all = gr.Button("顯示全部")

        table = gr.Dataframe(headers=df_from_tasks().columns.tolist(), interactive=False, row_count=10)
        logs_df = gr.Dataframe(headers=df_from_logs().columns.tolist(), interactive=False, row_count=8, label="工作日誌（自動產生）")

        with gr.Row():
            fig1 = gr.Plot(label="完成比例")
            fig2 = gr.Plot(label="任務番茄數")
            fig3 = gr.Plot(label="每日番茄趨勢")

        btn_query.click(search_and_filter, [q, status, priority, tag_kw, due_from, due_to], [table, fig1, fig2, fig3])
        btn_show_all.click(lambda: refresh_all(), None, [table, logs_df, fig1, fig2, fig3])

    # 新增 / 編輯 / 批次
    with gr.Tab("新增 / 編輯 / 批次"):
        gr.Markdown("### 新增任務")
        with gr.Row():
            i_name = gr.Textbox(label="名稱", placeholder="撰寫週會報告")
            i_desc = gr.Textbox(label="描述", lines=2)
        with gr.Row():
            i_priority = gr.Dropdown(PRIORITIES, value="中", label="優先度")
            i_due = gr.Textbox(label="到期日（YYYY-MM-DD，可空白）")
            i_est = gr.Number(label="預估番茄數", value=1, precision=0)
            i_tags = gr.Textbox(label="標籤（逗號分隔）", placeholder="OKR, Q4, 客訴")
        btn_add = gr.Button("新增 ▶", variant="primary")

        gr.Markdown("---\n### 編輯任務")
        with gr.Row():
            e_id = gr.Number(label="任務 ID", precision=0)
            e_name = gr.Textbox(label="名稱（可留空）")
        with gr.Row():
            e_desc = gr.Textbox(label="描述（可留空）", lines=2)
            e_priority = gr.Dropdown(PRIORITIES, value="中", label="優先度")
            e_due = gr.Textbox(label="到期日（YYYY-MM-DD，可空白）")
        with gr.Row():
            e_est = gr.Number(label="預估番茄數", precision=0)
            e_status = gr.Dropdown(STATUSES, value="未開始", label="狀態")
            e_tags = gr.Textbox(label="標籤（逗號分隔，可留空）")
        btn_edit = gr.Button("儲存修改 ✏️")

        gr.Markdown("---\n### 批次操作")
        with gr.Row():
            ids_str = gr.Textbox(label="ID 列表（逗號分隔）", placeholder="例如：1,2,5")
        with gr.Row():
            btn_bulk_done = gr.Button("批次標記完成 ✅")
            btn_bulk_del = gr.Button("批次刪除 🗑️")

        msg1 = gr.Markdown("")

        btn_add.click(add_task, [i_name, i_desc, i_priority, i_due, i_est, i_tags],
                      [table, logs_df, fig1, fig2, fig3, msg1])
        btn_edit.click(edit_task, [e_id, e_name, e_desc, e_priority, e_due, e_est, e_status, e_tags],
                       [table, logs_df, fig1, fig2, fig3, msg1])
        btn_bulk_done.click(bulk_done, [ids_str], [table, logs_df, fig1, fig2, fig3, msg1])
        btn_bulk_del.click(bulk_delete, [ids_str], [table, logs_df, fig1, fig2, fig3, msg1])

    # 番茄鐘
    with gr.Tab("番茄鐘"):
        with gr.Row():
            t_id = gr.Number(label="任務 ID", precision=0)
            focus_min = gr.Number(label="專注（分鐘）", value=25)
            short_b = gr.Number(label="短休（分鐘）", value=5)
            long_b = gr.Number(label="長休（分鐘）", value=15)
            long_every = gr.Number(label="每 N 顆長休", value=4, precision=0)
        with gr.Row():
            auto_next = gr.Checkbox(label="專注→休息→下一輪自動開始", value=True)
            sound_on = gr.Checkbox(label="階段結束提示音", value=True)
            auto_complete = gr.Checkbox(label="達到預估番茄時自動標記完成",
                                        value=STATE["settings"]["auto_complete_when_reach_est"])
        with gr.Row():
            btn_start = gr.Button("開始 ▶", variant="primary")
            btn_pause = gr.Button("暫停 ⏸️")
            btn_resume = gr.Button("繼續 ▶️")
            btn_stop = gr.Button("停止 ⏹️")

        timer_status = gr.Markdown("")
        timer_left = gr.Number(label="剩餘秒數", value=0, interactive=False)
        timer_bar = gr.HTML("")

        btn_start.click(start_timer, [t_id, focus_min, short_b, long_b, long_every, auto_next, sound_on],
                        [timer_status, timer_left, timer_bar])
        btn_pause.click(pause_timer, None, [timer_status, timer_left, timer_bar])
        btn_resume.click(resume_timer, None, [timer_status, timer_left, timer_bar])
        btn_stop.click(stop_timer, None, [timer_status, timer_left, timer_bar])
        auto_complete.change(toggle_auto_complete, [auto_complete], [timer_status])

    # Google Sheets 同步
    with gr.Tab("Google Sheets 同步"):
        gr.Markdown("把任務與番茄日誌同步到你指定的 Google 試算表（Colab 會跳出授權流程）。\n"
                    "系統使用兩個工作表：`tasks` 與 `logs`；若不存在會自動建立並補上標題列。")
        sheet_url = gr.Textbox(label="試算表 URL", value=GS["url"])
        auto_sync = gr.Checkbox(label="自動同步（每次任務/日誌變更後）", value=GS["auto_sync"])
        btn_connect = gr.Button("連線 / 建立工作表", variant="primary")
        btn_sync_tasks = gr.Button("立刻同步任務 ➜ tasks")
        btn_sync_logs = gr.Button("立刻同步日誌 ➜ logs")
        gs_msg = gr.Markdown("")

        def _connect(url, auto):
            GS["auto_sync"] = bool(auto)
            return gsheet_connect(url)

        def _set_auto(auto):
            GS["auto_sync"] = bool(auto)
            return f"⚙️ 自動同步：{'已開啟' if GS['auto_sync'] else '已關閉'}"

        btn_connect.click(_connect, [sheet_url, auto_sync], [gs_msg])
        auto_sync.change(_set_auto, [auto_sync], [gs_msg])
        btn_sync_tasks.click(lambda: gsheet_sync_tasks(), None, [gs_msg])
        btn_sync_logs.click(lambda: gsheet_sync_logs(), None, [gs_msg])
    # ✅ 關鍵：在 Blocks 內註冊 load 事件
    demo.load(lambda: refresh_all(), None, [table, logs_df, fig1, fig2, fig3])

# --- Blocks 區塊外 ---
# 開啟 generator queue (新版不需參數)
demo.queue()

# 在 Colab 內嵌顯示 & 同時提供公開網址
demo.launch(
    inline=True,
    share=True,
    show_error=True,
    server_name="0.0.0.0",
    server_port=7860
)



Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://760e738a636c186db0.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
